In [43]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn import preprocessing
from scipy import linalg
import matplotlib.pyplot as plt
from pykrige.uk import UniversalKriging

In [59]:
data = pd.read_csv('data/training_CNN_results_v3.csv')

In [60]:
clean_data = data[data["Name"] != 'Adafactor']

clean_data["Name"] = clean_data["Name"].replace(['SGD'],0.1)
clean_data["Name"] = clean_data["Name"].replace(['Adam'],0.3)
clean_data["Name"] = clean_data["Name"].replace(['RMSprop'],0.2)
clean_data["Name"] = clean_data["Name"].replace(['AdamW'],0.4)
clean_data["Name"] = clean_data["Name"].replace(['Adadelta'],0.5)
clean_data["Name"] = clean_data["Name"].replace(['Adagrad'],0.6)
clean_data["Name"] = clean_data["Name"].replace(['Adamax'],0.7)
clean_data["Name"] = clean_data["Name"].replace(['Nadam'],0.8)
clean_data["Name"] = clean_data["Name"].replace(['Ftrl'],0.9)

C:\Users\Lucas\AppData\Local\Temp\ipykernel_21264\2918968098.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data["Name"] = clean_data["Name"].replace(['SGD'],0.1)
C:\Users\Lucas\AppData\Local\Temp\ipykernel_21264\2918968098.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data["Name"] = clean_data["Name"].replace(['Adam'],0.3)
C:\Users\Lucas\AppData\Local\Temp\ipykernel_21264\2918968098.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [61]:
clean_data.to_csv('data/training_CNN_results_v3-1.csv', index=False)
clean_data.astype('float32')

,Name,Convoluted_Layers1,Convoluted_Filters1,Convoluted_Layers2,Convoluted_Filters2,Hidden_Layer1,Hidden_Layer2,Learning_Rate,Batch_Size,Loss
0,0.4,22.0,3.0,19.0,3.0,33.0,17.0,0.005487,63.0,0.039741
1,0.4,24.0,2.0,24.0,1.0,50.0,25.0,0.005466,127.0,0.051480
2,0.8,21.0,2.0,17.0,3.0,41.0,28.0,0.009989,99.0,0.043908
3,0.2,32.0,1.0,31.0,3.0,33.0,32.0,0.009013,78.0,0.074034
4,0.8,23.0,1.0,18.0,2.0,55.0,30.0,0.001348,106.0,0.068589
...,...,...,...,...,...,...,...,...,...,...
2138,0.7,22.0,1.0,29.0,1.0,57.0,25.0,0.002672,45.0,0.256532
2139,0.3,22.0,3.0,21.0,1.0,36.0,29.0,0.009747,67.0,0.062995
2140,0.6,29.0,3.0,19.0,3.0,40.0,32.0,0.000519,85.0,1.668306
2141,0.4,23.0,1.0,32.0,3.0,46.0,32.0,0.004172,43.0,0.049501


In [62]:
halfData = clean_data.sample(frac = 0.5)
otherHalfData = clean_data.drop(clean_data2.index)
clean_data = halfData


In [63]:
print(halfData)

      Name  Convoluted_Layers1  Convoluted_Filters1  Convoluted_Layers2  \
2026   0.8                  19                    3                  28   
1584   0.4                  16                    1                  31   
308    0.2                  32                    1                  21   
1297   0.4                  26                    1                  28   
1707   0.4                  26                    2                  20   
...    ...                 ...                  ...                 ...   
2098   0.2                  32                    2                  32   
1996   0.6                  17                    2                  16   
2028   0.5                  31                    1                  29   
1349   0.2                  26                    1                  29   
997    0.8                  24                    2                  29   

      Convoluted_Filters2  Hidden_Layer1  Hidden_Layer2  Learning_Rate  \
2026                    2

In [64]:
loss_data = clean_data["Loss"]
loss_data.astype('float32')
loss_data = loss_data.to_numpy()

In [65]:
min_loc = np.argmin(loss_data)

In [80]:
np.set_printoptions(suppress=True)
all_data = otherHalfData.to_numpy()

In [67]:
data = halfData.drop('Loss', axis=1)
dataOnly = np.asarray(data)

In [68]:
print(data)

      Name  Convoluted_Layers1  Convoluted_Filters1  Convoluted_Layers2  \
2026   0.8                  19                    3                  28   
1584   0.4                  16                    1                  31   
308    0.2                  32                    1                  21   
1297   0.4                  26                    1                  28   
1707   0.4                  26                    2                  20   
...    ...                 ...                  ...                 ...   
2098   0.2                  32                    2                  32   
1996   0.6                  17                    2                  16   
2028   0.5                  31                    1                  29   
1349   0.2                  26                    1                  29   
997    0.8                  24                    2                  29   

      Convoluted_Filters2  Hidden_Layer1  Hidden_Layer2  Learning_Rate  \
2026                    2

In [69]:
mn = np.append(dataOnly[min_loc],loss_data[min_loc])

In [70]:
kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)

In [71]:
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=True)

In [81]:
gp.fit(dataOnly, loss_data)

GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=1, nu=1.5),
                         n_restarts_optimizer=5, normalize_y=True)

In [82]:
from scipy.stats import norm

In [83]:
surrogate_values = []
y_mean_values = []
y_cov_values = []

In [84]:
for X in range (len(all_data)):
    y_mean, y_cov = gp.predict(all_data[X:X+1,:-1], return_cov=True)
    y_mean_values.append(y_mean[0])
    y_cov_values.append(y_cov[0][0])
    dist = norm(loc=0.0,scale=1.0)
    sigma_tilde = (y_cov[0][0])**(1/2)
    u = (loss_data[min_loc] - y_mean[0])/sigma_tilde
    ei = sigma_tilde*((u * dist.cdf(u))+dist.pdf(u))
    surrogate_values.append(-ei)

In [86]:
print(surrogate_values)
print(y_cov_values)
print(y_mean_values)

[-0.1266939564468725, -0.0, -0.12665806823887008, -0.0, -0.12661448828988783, -0.0, -0.12670354154778168, -0.0, -0.0, -0.0, -0.12663956973832097, -0.12670711088498932, -0.0, -0.12715738514699698, -0.0, -0.12665704850853957, -0.1265959195792954, -0.1267129106090529, -0.0, -0.0, -0.12671874571656278, -0.12889405973694318, -0.12668578384139642, -0.0, -0.1267039441764405, -0.0, -0.12638178154706062, -0.0, -0.12742426461000533, -0.12671167086625493, -0.12668011642019564, -0.12670568209001173, -0.0, -0.12607594886525636, -0.12682277326970215, -0.0, -0.126986605056489, -0.0, -0.0, -0.0, -0.1266568553140756, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.12672794636833046, -0.12672022990157517, -0.0, -0.12666034828452027, -0.12665808383120722, -0.1232614157372911, -0.12665646682544215, -0.12701239559417019, -0.12574866651325028, -0.1263884538420025, -0.1266509764825408, -0.12664935068512978, -0.12671329205121734, -0.0, -0.0, -0.12666006829626394, -0.1266612304811714